In [1]:
from gazpacho import get, Soup
import numpy
import pandas
import requests, bs4
import re, os
import time

In [2]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://www.whitelabs.com/yeast-single?id=102&type=YEAST'

# fetch step
html = get(url)

# extract step
soup = Soup(html)

In [3]:
# use find() to parse a specific HTML
gaz_data = soup.find('div', {'itemprop':'address'})
gaz_data

In [4]:
def pullDescription(soup):
    description = soup.find('div', {'itemprop':'address'})
    print(description)
    x = str(description)
    start = x.find('<p>')
    end = x.find('<br>')
    description = x[start+3:end]
    return description

In [5]:
pullDescription(soup)

None


'n'

In [6]:
# narrow things down to the table
gaz_table = soup.find('div', {'class':'accordion-style'})
gaz_table

<div class="accordion-style" id="accordionEx" role="tablist" aria-multiselectable="true">
  <ul class="beer_style">
    <li>
American IPA
</li>
    <li>
Blonde Ale
</li>
    <li>
Brown Ale
</li>
    <li>
Double IPA
</li>
    <li>
English Bitter
</li>
    <li>
English IPA
</li>
    <li>
Hazy/Juicy IPA
</li>
    <li>
Imperial Stout
</li>
    <li>
Old Ale
</li>
    <li>
Pale Ale
</li>
    <li>
Porter
</li>
    <li>
Red Ale
</li>
    <li>
Stout
</li>
  </ul>
</div>

In [7]:
def pullStyle(soup):
    a = (str(gaz_table.html))
    a = a.replace('<li>\n' , '')
    a = a.replace('\n</li>\n', '')
    a = a.replace('</ul>\n</div>', '')
    x = len(a)
    #print(x)
    style = a[120:]
    #print(a)
    sytle = style.split("   ")
    style = sytle[:]
    return style

In [8]:
pullStyle(soup)

['American IPA',
 ' Blonde Ale',
 ' Brown Ale',
 ' Double IPA',
 ' English Bitter',
 ' English IPA',
 ' Hazy/Juicy IPA',
 ' Imperial Stout',
 ' Old Ale',
 ' Pale Ale',
 ' Porter',
 ' Red Ale',
 ' Stout  ']

In [9]:
def pullAttenuation(soup):
    metrics = soup.find('p')
    clean_metrics =[]
    for i in metrics:
        i = str(i).replace('</p>', '')
        i = str(i).replace('<p>', '')
        #i = str(i).replace("\\r\\n", '')
        i = re.sub('\s+', '', i)
        clean_metrics.append(i)
    attenuation = clean_metrics[0]
    
    return attenuation

In [10]:
pullAttenuation(soup)

'63%-70%'

In [11]:
def pullFlocculation(soup):
    metrics = soup.find('p')
    clean_metrics =[]
    for i in metrics:
        i = str(i).replace('</p>', '')
        i = str(i).replace('<p>', '')
        #i = str(i).replace("\\r\\n", '')
        #i = re.sub('\s+', '', i)
        clean_metrics.append(i)
    flocculation = clean_metrics[2]
    
    return flocculation

In [12]:
pullFlocculation(soup)

'Very High'

In [13]:
def pullABV(soup):
    metrics = soup.find('p')
    clean_metrics =[]
    for i in metrics:
        i = str(i).replace('</p>', '')
        i = str(i).replace('<p>', '')
        #i = str(i).replace("\\r\\n", '')
        #i = re.sub('\s+', '', i)
        clean_metrics.append(i)
    alcohol_tolerance = clean_metrics[3]
    
    return alcohol_tolerance

In [14]:
pullABV(soup)

'Medium (5-10%)'

In [15]:
def pullTEMP(soup):
    metrics = soup.find('p')
    clean_metrics =[]
    for i in metrics:
        i = str(i).replace('</p>', '')
        i = str(i).replace('<p>', '')
        i = str(i).replace('<br>', ' ')
        i = re.sub('\s+', '', i)
        i = str(i).replace('F', 'F ')
        #i = str(i).replace("\\r\\n", '')
        
        clean_metrics.append(i)
    fermentation_temperature = clean_metrics[4]
    
    return fermentation_temperature

In [16]:
pullTEMP(soup)

'65°-68°F 18°-20°C'

In [17]:
def pullSTA1(soup):
    metrics = soup.find('p')
    clean_metrics =[]
    for i in metrics:
        i = str(i).replace('</p>', '')
        i = str(i).replace('<p>', '')
        i = str(i).replace('<br>', ' ')
        i = re.sub('\s+', '', i)
        i = str(i).replace('F', 'F ')
        #i = str(i).replace("\\r\\n", '')
        
        clean_metrics.append(i)
    STA1 = clean_metrics[5]
    
    return STA1

In [18]:
pullSTA1(soup)

'Negative'

In [19]:
def pullNumber(soup):
    number = soup.find('ul', {'class' :'details'})
    number = str(number)
    ns = number.find('WLP')
    number = number[ns:ns+6]
    return number

In [20]:
pullNumber(soup)

'WLP002'

In [21]:
    description = soup.find('div', {'class':'dynamic-content'})

In [22]:
def pullStrain(soup):
    strain = soup.find('h1')
    strain = str(strain).replace('<h1>', '')
    strain = str(strain).replace('</h1>', '')
    return strain

In [23]:
pullStrain(soup)

'English Ale Yeast'

In [24]:
import pandas as pd

def loadTable(soup):
    style = pullStyle(soup)
    #pullMetrics(soup)
    number = pullNumber(soup)
    strain = pullStrain(soup)
    attenuation = pullAttenuation(soup)
    flocculation = pullFlocculation(soup)
    alcohol_tolerance = pullABV(soup)
    fermentation_temperature = pullTEMP(soup)
    STA1 = pullSTA1(soup)
    description = pullDescription(soup)
    entry = {'Number' : [number], 'Strain': [strain],
        'Attenuation' : [attenuation],
        'Flocculation' : [flocculation],
        'Alcohol_Tolerance' : [alcohol_tolerance],
        'Fermentation_Temperature' : [fermentation_temperature],
        'STA1' : [STA1],
        'Description' : [description]}
    df = pd.DataFrame(entry)
    return df

In [25]:
df = loadTable(soup)

None


# Pulling all the urls that are needed

In [26]:
url_list = []
# https://www.whitelabs.com/yeast-single?id=105&type=YEAST

url_1 = 'https://www.whitelabs.com/yeast-single?id='
url_2 = '&type=YEAST'

i = 101

while i < 400:
    full_url = url_1 + str(i) + url_2
    url_list.append(full_url)
    i = i+1
    #print(full_url)

In [27]:
print(len(url_list))

299


In [28]:
for i in url_list:
    print(i)

https://www.whitelabs.com/yeast-single?id=101&type=YEAST
https://www.whitelabs.com/yeast-single?id=102&type=YEAST
https://www.whitelabs.com/yeast-single?id=103&type=YEAST
https://www.whitelabs.com/yeast-single?id=104&type=YEAST
https://www.whitelabs.com/yeast-single?id=105&type=YEAST
https://www.whitelabs.com/yeast-single?id=106&type=YEAST
https://www.whitelabs.com/yeast-single?id=107&type=YEAST
https://www.whitelabs.com/yeast-single?id=108&type=YEAST
https://www.whitelabs.com/yeast-single?id=109&type=YEAST
https://www.whitelabs.com/yeast-single?id=110&type=YEAST
https://www.whitelabs.com/yeast-single?id=111&type=YEAST
https://www.whitelabs.com/yeast-single?id=112&type=YEAST
https://www.whitelabs.com/yeast-single?id=113&type=YEAST
https://www.whitelabs.com/yeast-single?id=114&type=YEAST
https://www.whitelabs.com/yeast-single?id=115&type=YEAST
https://www.whitelabs.com/yeast-single?id=116&type=YEAST
https://www.whitelabs.com/yeast-single?id=117&type=YEAST
https://www.whitelabs.com/yeast

In [29]:
df1 = df

In [30]:
for i in url_list:
    url = str(i)
    print(url)

    # fetch step
    html = get(i)
    time.sleep(5)
    
    # extract step
    soup = Soup(html)
    
    # feed soup into loadTable
    try:
        table = loadTable(soup)
    except IndexError:
        print('ignored')

    df1 = pd.concat([df1, table])

https://www.whitelabs.com/yeast-single?id=101&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=102&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=103&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=104&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=105&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=106&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=107&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=108&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=109&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=110&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=111&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=112&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=113&type=YEAST
ignored
https://www.whitelabs.com/yeast-single?id=114&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=115&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=116&type=YEAST
None
https

None
https://www.whitelabs.com/yeast-single?id=234&type=YEAST
None
https://www.whitelabs.com/yeast-single?id=235&type=YEAST


HTTPError: 500 - Internal Server Error

In [ ]:
df1

In [ ]:
df1.to_csv('White_Labs_Strains_with_number.csv')